In [3]:
import numpy as np
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append("..")
from utils import *
import warnings
warnings.filterwarnings("ignore")
import os
import psutil

pd.options.display.max_columns=1000
pd.options.display.max_rows=1000

import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score 
from catboost import CatBoostClassifier, Pool

In [4]:
''' Data Load '''
train = pl.read_csv('/home/workspace/DACON/Click_predict/data/train.csv')
test = pl.read_csv('/home/workspace/DACON/Click_predict/data/test.csv')
train, test = to_pandas(train, test)
# train, test = load_data()
# train = sampling('/home/workspace/DACON/Click_predict/data/train.csv')
# test = pd.read_csv('/home/workspace/DACON/Click_predict/data/test.csv')

''' Memory Optimization '''
# train = reduce_mem_usage(train)
# test = reduce_mem_usage(test)
# print("k")

''' Preprocessing '''
train = preprocessing(train)
test = preprocessing(test, is_test = True)


KeyboardInterrupt: 

In [ ]:
''' Train / Vaild Split'''
X_train = train.drop(columns = ["Click"]) 
y_train = train["Click"]
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)
category_cols = list(X_train.select_dtypes(include=['category']).columns)

In [ ]:
catb = CatBoostClassifier()
catb.load_model('/home/workspace/DACON/Click_predict/Model/best_model3.cb')

In [ ]:
catb.feature_importances_

In [ ]:
# 전처리 전에 best: 0.7682916 (100)
# 전처리 후에 best: 0.7684399 (100)
y_pred = catb.predict_proba(X_valid)[:,1]
# ROC AUC 평가
score = roc_auc_score(y_valid, y_pred)
print(score)

0.7745402543496831


In [ ]:
pred = catb.predict_proba(test)

In [ ]:
sample_submission = pd.read_csv('/home/workspace/DACON/Click_predict/data/sample_submission.csv')
sample_submission['Click'] = pred
sample_submission.to_csv('click_ratio(catb2).csv',index=False)

OSError: Cannot save file into a non-existent directory: '/home/sgh/yes/envs/DACON'

In [ ]:
# 시스템 메모리 정보 가져오기
mem = psutil.virtual_memory()

# 남은 램 용량 확인
available_memory = mem.available / (1024 ** 3)  # GB 단위로 변환
print("Available Memory:", available_memory, "GB")

Available Memory: 40.082908630371094 GB


# Models

## Catboost

In [ ]:
from catboost import CatBoostClassifier, Pool

train_pool = Pool(X_train1, y_train1 ,cat_features = categorical_feat1)
val_pool = Pool(X_test1, y_test1, cat_features = categorical_feat1)


catb_params = {
    'iterations' : 200,
    'learning_rate': 0.05 ,
    "max_depth": 10,
    'eval_metric' :'AUC', #GPU 이용시 AUC말고 다른 매트릭으로 평가됨 
    'task_type' : 'GPU', #device는 굳이 전달 안 해도 None이 디폴트로, 자동으로 지정된다.
    'verbose': 1,
    'random_seed' : 721,
    'gpu_ram_part': 0.9 #gpu의 80%만 사용하라. default = 0.95 (95%)
    #'used_ram_limit': '20gb' #cpu가 20gb만 이용하도록 함
    }

catb = CatBoostClassifier(**catb_params)
catb.fit(train_pool, eval_set=val_pool, verbose=True)

# 예측 확률 계산
#y_pred = catb.predict_proba(X_test1)[:,1]

# ROC AUC 평가
#score = roc_auc_score(y_test1, y_pred)
#print(score)

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7243258	best: 0.7243258 (0)	total: 2.91s	remaining: 9m 40s
1:	total: 6.15s	remaining: 10m 9s
2:	total: 9.14s	remaining: 10m
3:	total: 12.7s	remaining: 10m 24s
4:	total: 15.9s	remaining: 10m 21s
5:	test: 0.7363940	best: 0.7363940 (5)	total: 19.2s	remaining: 10m 21s
6:	total: 22.2s	remaining: 10m 12s
7:	total: 25.9s	remaining: 10m 22s
8:	total: 29.2s	remaining: 10m 20s
9:	total: 32.9s	remaining: 10m 24s
10:	test: 0.7396747	best: 0.7396747 (10)	total: 36.4s	remaining: 10m 25s
11:	total: 39.8s	remaining: 10m 23s
12:	total: 43.1s	remaining: 10m 19s
13:	total: 46.2s	remaining: 10m 14s
14:	total: 48.9s	remaining: 10m 3s
15:	test: 0.7426370	best: 0.7426370 (15)	total: 52.1s	remaining: 9m 58s
16:	total: 55.4s	remaining: 9m 56s
17:	total: 58.7s	remaining: 9m 53s
18:	total: 1m 1s	remaining: 9m 47s
19:	total: 1m 4s	remaining: 9m 41s
20:	test: 0.7459271	best: 0.7459271 (20)	total: 1m 8s	remaining: 9m 41s
21:	total: 1m 11s	remaining: 9m 35s
22:	total: 1m 14s	remaining: 9m 32s
23:	total: 1

In [ ]:
import joblib

# 모델 저장 경로
model_path = "/home/sgh/yes/envs/DACON/models/"

# 경로가 존재하지 않으면 생성
os.makedirs(model_path, exist_ok=True)

# 모델 저장
catb.save_model(os.path.join(model_path, "click_ratio_catb(vanilla).pth"))

In [ ]:
print(X_test.columns)
print(X_train1.columns)

Index(['F02', 'F04', 'F06', 'F07', 'F08', 'F09', 'F10', 'F11', 'F12', 'F13',
       'F14', 'F16', 'F17', 'F18', 'F19', 'F21', 'F22', 'F23', 'F24', 'F25',
       'F28', 'F30', 'F31', 'F32', 'F33', 'F34', 'F35', 'F36', 'F37', 'F38',
       'F39'],
      dtype='object')
Index(['F02', 'F04', 'F06', 'F07', 'F08', 'F09', 'F10', 'F11', 'F12', 'F13',
       'F14', 'F16', 'F17', 'F18', 'F19', 'F21', 'F22', 'F23', 'F24', 'F25',
       'F28', 'F30', 'F31', 'F32', 'F33', 'F34', 'F35', 'F36', 'F37', 'F38',
       'F39'],
      dtype='object')


In [ ]:
#sample_submission = sample_submission.to_pandas()

y_submission_pred2 = catb.predict_proba(X_test)[:,1]

sample_submission['Click'] = y_submission_pred2
submission = sample_submission.copy()

submission.to_csv('/home/sgh/yes/envs/DACON/click_ratio(catb2).csv',index=False)